# Import Library

In [1]:
# Install packages
%pip install -U scikit-learn
%pip install ftfy
%pip install optuna


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip3.9 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip3.9 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip3.9 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Imports

# from google.colab import drive
import os

import traceback
import sys
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
import optuna
from optuna.trial import TrialState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

from ftfy import fix_text
import warnings
warnings.simplefilter("ignore", UserWarning)

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import math

# Set Path

In [3]:

prj_path = '/home/mlworker/Quang/HealthCare/Source_14012023_v4/'
data_path = prj_path + "/data/"
prj_path_opt= prj_path + "optimize_hyperparam/opt_results/opt_results_12022023_v4/"

os.chdir(prj_path)

# Create Dict data for all cities

In [4]:
cities = [
     'Bắc Giang', 'Cao Bằng', 'Cà Mau',
        'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh', 'Hòa Bình',
        'Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
        'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
        'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
        'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
        'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La',
          'TT Huế',
        'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
        ]

def get_dict_all_city_data():
  cities_data = {}  
  for city in cities:
    city_result = pd.read_excel(prj_path+'data/new_data/DH/squeezed/squeezed_'+city+'.xlsx')  
    # Đoạn này rất quan trọng. Vì việc optimize không được đụng vào 24 tháng (2016-2017) để dự báo. 
    # Dữ liệu optimize tính từ 1997- 30/12/2015. Sau đó tách ra train và test trên bộ này.
    # lọc 2 năm cuối ra khỏi bộ dữ liệu trước khi chạy optimize 
    # đoạn này áp dụng cho tất cả các bước optimize trong project
    city_result = city_result.loc[city_result['year_month'] < '2013-1-1'] 
    cities_data[city] = city_result
  return cities_data

dict_full_data = get_dict_all_city_data()

# Seed and Configuration

In [5]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(42)

In [6]:
# Set hyperparameters as args using the Configuration class
class Configuration():
    def __init__(self):
        self.test_size = 24
        self.look_back = 3
        self.n_predicted_month = 3
        self.n_features = 3
        self.seed = 42
        self.batch_size = 16
        self.device = torch.device("cuda")
        self.epochs = 300

args = Configuration()

# Pre-processing Functions

In [7]:
# Define data (pre-)processing functions
# modification
def get_city_data(city_name):
    """Returns Diarrhoea rate and climate data""" 
    city_data = dict_full_data[city_name].drop(columns=['Diarrhoea_cases', 'province', 'year_month',
                                                        'Influenza_rates','Dengue_fever_rates',
                                                        'Influenza_cases','Dengue_fever_cases', 'year', 'month'], 
                                                                  axis=1, 
                                                                  inplace=False)    
    return city_data


def convert_to_stationary(city_data):
    """Subtracts previous value for all cols except disease rates"""
    for col_name in city_data.columns:
        if col_name != 'Diarrhoea_rates':
            try:
                city_data[col_name] = city_data[col_name] - city_data[col_name].shift()
            except:
                print(col_name)
    return city_data

def impute_missing_value(city_data):
    """
    Cơ bản dữ liệu bị thiếu sót rất nhiều: Như Điện Biên 1997 -2003 là thiếu dữ liệu về bệnh
    Hàm này sẽ tự sinh ra dữ liệu bị thiếu. Nếu tháng nào không có số liệu thì tính như sau:
    12 tháng đầu không có số liệu thì gán = 0
    tháng 13-24 không có số liệu, sẽ lấy giá trị của tháng cùng kỳ năm trước
    tháng từ 24 trở đi sẽ lấy giá trị nhỏ nhất của 2 tháng cùng kỳ trong 2 năm gần nhất.
    Do Điện Biên bằng 0 nên sau khi xử lý từ 1997 -2003 là đều = 0.  
    """
    for col in city_data.columns:
        for index in range(len(city_data[col])):
            if np.isnan(city_data[col].iloc[index]):
                if index < 12:
                    city_data[col].iloc[index] = 0
                elif index >= 12 and index <= 24:
                    city_data[col].iloc[index] = city_data[col].iloc[index - 12]
                else:
                    city_data[col].iloc[index] = min(city_data[col].iloc[index - 12], city_data[col].iloc[index - 24])
    return city_data

In [8]:
def split_data(data, look_back ):
    """Splits data into train and test sets based on args (Configuration class)"""
    train = data[: -args.test_size]
    print('lookback', look_back)
    test = data[-args.test_size - look_back: ]
    return train, test

In [9]:
def to_supervised(data, d_in=args.look_back, d_out=args.n_predicted_month, features_list=[]):
    """
    Frames time-series as supervised learning dataset.
    
    Args:
      d_in: lookback window
      d_out: number of predicted months
      features_list: list of all features **where last col is the disease incidence**

    Returns:
      Numpy arrays of disease incidence (y) and other predictors (X)
    """
    X, y = list(), list()
    for index, _ in enumerate(data):
        in_end = index + d_in
        out_end = in_end + d_out
        if out_end <= len(data):
            if len(features_list) == 0 :
                X.append(data[index: in_end, :])
            else:
                X.append(data[index: in_end, features_list])
            y.append(data[in_end: out_end, -1])
    return np.array(X), np.array(y)

In [10]:
def select_feature(train, specific_data):
    """Selects args.n_features top features using RFE"""
    train_X, train_y = to_supervised(train, d_in=1, d_out=1)
    train_X, train_y = np.squeeze(train_X), np.squeeze(train_y)
    rfe = RFE(RandomForestRegressor(n_estimators=500, random_state=args.seed), n_features_to_select=args.n_features)
    fit = rfe.fit(train_X, train_y)
    important_features = list()
    # print("Important Feature:")
    for i in range(len(fit.support_)):
        if fit.support_[i]:
            important_features.append(i)
            # print(specific_data.columns[i])
    return np.array(important_features)

In [11]:
def get_data(train_np, test_np, batch_size, specific_data):
    """
    Returns important feature list and data formatted for input into Pytorch 
    models
    """
    important_features = select_feature(train_np, specific_data)

    train_X, train_y = to_supervised(train_np, features_list=important_features)
    test_X, test_y = to_supervised(test_np, features_list=important_features)
    train_tensor = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
    test_tensor = (torch.from_numpy(test_X), torch.from_numpy(test_y))

    train_loader = DataLoader(train_tensor, batch_size=batch_size, shuffle=True)

    return important_features, train_loader, test_tensor

In [12]:
#Define Pytorch LSTM model
class MultiVariateLSTM(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature, hidden_size=hidden_size, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, args.n_predicted_month)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
    
    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = output[:, -1, :]
        y_predicted = self.linear(last_hidden_vector)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            #return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted
    
    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

#Define Pytorch LSTM-ATT model
class MultiVariateLSTM_Attention(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM_Attention, self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature, hidden_size=hidden_size, num_layers=n_layers, batch_first=True)
        self.attention_linear = nn.Linear(hidden_size, hidden_size)
        # self.linear = nn.Linear(hidden_size*2, args.n_predicted_month)
        self.linear = nn.Linear(hidden_size, args.n_predicted_month)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
    
    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = last_hidden[-1]
        remain_hidden_vector = output
        e_t = remain_hidden_vector.bmm(self.attention_linear(last_hidden_vector).unsqueeze(2)).squeeze(-1)
        alpha_t = F.softmax(e_t, dim=1)
        attenion_vector = remain_hidden_vector.transpose(2, 1).bmm(alpha_t.unsqueeze(2)).squeeze(-1)
        # combine_vector = torch.cat((last_hidden_vector, attenion_vector), dim=1)
        # combine_vector = last_hidden_vector + attenion_vector
        y_predicted = self.linear(attenion_vector)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted
    
    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

# Define Pytorch CNN model
class MultivariateCNN(nn.Module):
    def __init__(self, num_filters=[100, 100, 100], dropout=0.01):
        super(MultivariateCNN, self).__init__()
        self.loss_fn = loss = nn.MSELoss()
        self.filter_sizes = [1, 2, 3]
        self.conv1d_list = nn.ModuleList([nn.Conv1d(args.n_features, num_filters[i], self.filter_sizes[i]) for i in range(len(self.filter_sizes))])
        self.linear = nn.Linear(np.sum(num_filters), args.n_predicted_month)
        self.dropout = nn.Dropout(dropout)
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))

    def forward(self, X_batch, y_batch=None):
        X_batch = X_batch.permute(0, 2, 1)  #(batch_size, n_features, n_look_back)
        X_conv_list = [F.relu(conv1d(X_batch)) for conv1d in self.conv1d_list]
        X_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2]) for x_conv in X_conv_list]
        X_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in X_pool_list], dim=1)
        y_predicted = self.linear(self.dropout(X_fc))
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted

    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

# Define Transformer Model

In [13]:
# Define Pytorch Transformer model
class PositionalEncoder(nn.Module):
    def __init__(self, d_model=3, n_feature=3, dropout=0.1):
        super().__init__()
        
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(n_feature, d_model)
        for pos in range(n_feature):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos/(10000**(2*i/d_model)))
                if i + 1 < d_model:
                    pe[pos, i+1] = math.cos(pos/(10000**((2*i+1)/d_model)))
        pe = pe.unsqueeze(0)        
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x*math.sqrt(self.d_model)
        length = x.size(1)
        pe = Variable(self.pe[:, :length], requires_grad=False)
        if x.is_cuda:
            pe.cuda()
        x = x + pe
        x = self.dropout(x)
        
        return x

class TransformerModel(nn.Module):
    def __init__(self, d_input=3, n_head=3, hidden_size=256, n_layers=3, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.hidden_size = hidden_size
        self.pe = PositionalEncoder(dropout=dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_input, nhead=n_head, dim_feedforward=hidden_size, dropout=dropout, activation='gelu')
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        self.decoder = nn.Linear(d_input*n_head, args.n_predicted_month)
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
        self.loss_fn = nn.MSELoss()
    
    def forward(self, X_batch, y_batch=None):
        X_batch = self.pe(X_batch)
        X_batch = self.transformer_encoder(X_batch)
        X_batch = X_batch.view(X_batch.size(0), -1)
        
        y_predicted = self.decoder(X_batch)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
        else:
            return y_predicted
        return X_batch
    
    def predict(self, X):
        with torch.no_grad():
            X = torch.tensor(X, device=args.device)
        return self.forward(X).squeeze()

# Class Trainer

In [14]:
# Create class to train and evaluate models
class Trainer():
    def __init__(self, city, model_type, learning_rate, important_features, train_loader, test_tensor, n_layers=2, hidden_size=128, num_filters=[100, 100, 100], dropout=0.01):
        """
        Initialise trainer, allowing input of LSTM, LSTM-ATT, or CNN 
        hyperparameters. Adam optimiser used for all models.
        """
        self.model_type = model_type    
        self.city = city    
        self.Model = self.init_model(model_type, n_layers, hidden_size, num_filters, dropout, city)
        self.Model.double().to(args.device)
        self.optimizer = torch.optim.Adam(self.Model.parameters(), lr=learning_rate)
        self.important_features, self.train_loader, self.test_tensor = important_features, train_loader, test_tensor
    
    def init_model(self, model_type, n_layers, hidden_size, num_filters, dropout, city):
        """Initialise a model based on whether LSTM, LSTM-ATT, CNN or Transformer is chosen."""
        model = TransformerModel()
        if model_type.lower() == 'lstm':
            model = MultiVariateLSTM(args.n_features, n_layers, hidden_size)
        elif model_type.lower() == 'lstm_attention':
            model = MultiVariateLSTM_Attention(args.n_features, n_layers, hidden_size)
        elif model_type.lower() == 'cnn':
            model = MultivariateCNN(num_filters, dropout)
        elif model_type.lower() == 'transformers':
            model = TransformerModel(d_input=args.look_back, n_head=3, hidden_size=hidden_size, n_layers=n_layers, dropout=dropout)
        return model

    def step(self, batch):
        self.Model.train()
        X_batch, y_batch = tuple(t.to(args.device) for t in batch)
        self.optimizer.zero_grad()
        y_pred, loss = self.Model.forward(X_batch, y_batch)
        loss.backward()
        self.optimizer.step()
        return loss.mean().item()

    def validation(self):
        self.Model.eval()
        eval_loss = 0.0

        result = {}

        y_true = np.array([])
        y_pred = np.array([])

        X_batch, y_batch = tuple(t.to(args.device) for t in self.test_tensor)
        with torch.no_grad():
            outputs, loss = self.Model.forward(X_batch, y_batch)
            eval_loss = loss.mean().item()

        return eval_loss

    def train(self, epochs=20):
        best_lost = float("inf")
        best_model = None
        for epoch in range(epochs):
            total_loss = 0.0
            for batch in self.train_loader:
                loss = self.step(batch)
                total_loss += loss
            train_loss = total_loss/len(self.train_loader)
            eval_loss = self.validation()
            if eval_loss < best_lost:
                best_lost = eval_loss
                best_model = copy.deepcopy(self.Model)
            if (epoch + 1) == epochs or (epoch + 1) in [c + 1 for c in range(epochs) if c % int(epochs/4) == 0]:
                print(f"Epoch: {epoch:2}/{epochs:2} - train_loss: {train_loss:.4f} - test_loss: {eval_loss:4f}")
        self.Model = best_model
        self.Model.eval()
        return None

    # Lưu model vào trong thư mục models
    def save_model_to(self, path = '', city =''):       
        torch.save(self.Model, path)

    def load_model_to(self, path = ''):       
        return torch.load(path)

    def evaluate_model(self, np_data=None, plot=True, scaled=True, city=None, k_steps=None, y_scaler =None):
        assert scaled, "data must be scaled"
        self.Model.eval()
        tensor_data = torch.from_numpy(np_data)
        rmse_list = []
        mae_list = [] 
        mape_list = []

        y_predicted_list = []
        y_true_list = []

        for k_steps in range(1, args.n_predicted_month + 1):
            y_predicted = []
            y_true = []
            for index in range(tensor_data.size(0) - args.look_back):
                X = tensor_data[index: index + args.look_back, self.important_features]
                # yhat = self.Model.predict(X.unsqueeze(0)).squeeze()

                yhat = self.Model.predict(X.unsqueeze(0))
                yhat = yhat.squeeze()

                y_predicted.append(yhat.detach().cpu().numpy()[k_steps - 1])
                y_true.append(tensor_data[index + args.look_back, -1].detach().cpu().numpy())

            y_predicted = y_scaler.inverse_transform(np.array(y_predicted).reshape(-1, 1)).reshape(-1, )
            y_true = y_scaler.inverse_transform(np.array(y_true).reshape(-1, 1)).reshape(-1, )

            rmse = mean_squared_error(y_true, y_predicted, squared=False)
            mae = mean_absolute_error(y_true, y_predicted)
            mape = mean_absolute_percentage_error(y_true, y_predicted)

            rmse_list.append(rmse)
            mae_list.append(mae)
            mape_list.append(mape)

            # print('City: '+self.city+'  _algo:'+self.model_type+'  -RMSE: '+str(rmse))
            if plot==True:
              plt.grid(True)
              plt.plot(y_predicted, label='predicted')
              plt.plot(y_true, label='actual')
              plt.title(f"k-steps = {k_steps} - city: "+self.city+'  _algo:'+self.model_type+'  -RMSE: '+str(rmse))
              plt.legend()
              plt.show()

              plt.show()
            y_predicted_list.append(y_predicted)
            y_true_list.append(y_true)

        return y_true_list, y_predicted_list, rmse_list, mae_list, mape_list

# Objective Function

In [15]:
def objective(trial, city):
    # Define search parameters
    n_layers = trial.suggest_int('n layers', 3, 6) # a    
    hidden_size = trial.suggest_int('Hidden size', 5, 384, log=True) #b
    learning_rate = trial.suggest_loguniform('Learning rate', 1e-4, 1e-2)
    dropout = trial.suggest_uniform('Dropout rate', 0.01, 0.80)
    args.epochs = trial.suggest_int('Epochs', 100, 500, step=10)
    lookback_window = 3 # fix cứng optimize sau

    # Pre-process data
    specific_data = get_city_data(fix_text(city))
    specific_data = impute_missing_value(specific_data)
    specific_data = convert_to_stationary(specific_data)
    specific_data.dropna(inplace=True)

    train, test = split_data(specific_data,lookback_window)

    # Fit data scaler to training data
    full_scaler = MinMaxScaler().fit(train)
    y_scaler = MinMaxScaler().fit(train.values[:, -1].reshape(-1, 1))

    # Scale train and test data
    train = full_scaler.transform(train)
    test = full_scaler.transform(test)

    # Get data to run model
    important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, specific_data)

    # Transformer model
    trainer = Trainer(model_type='transformer',
                  city = city,
                  important_features=important_features,
                  train_loader=train_loader,
                  test_tensor=test_tensor,
                  n_layers=n_layers,
                  hidden_size=hidden_size,
                  learning_rate=learning_rate,
                  dropout=dropout)

    # Train model
    # trainer.train(epochs=args.epochs, trial=trial)
    trainer.train(epochs=args.epochs)

    # Evaluate model
    y_true, y_pred, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= False, scaled=True, city=city, y_scaler = y_scaler)
    # _, _, rmse, mae, = trainer.evaluate_model(np_data=test, plot=False, scaled=True, city=city, y_scaler=y_scaler)

    return mae_list[0]    

# Main Cell For Optimize

In [ ]:
#########################
# Main Cell for optimize
#########################
dt_started = datetime.now()

##################################################
# Input param for Optimize Run
ntry = 50
njob = -1

# Lưu thông tin traceback study và error city trong quá trình optimize
l_study_city ={}
# l_errCity =[]

if __name__ == '__main__':  
  best_param = pd.DataFrame()
  l_studies = {}
  l_errCity =[]

  for city in cities:
    # Use Tree-structured Parzen Estimator sampler to minimise RMSE
    sampler = optuna.samplers.TPESampler()
    study = optuna.create_study(sampler=sampler, direction='minimize')

    # truyền multiple param vào trong biến trial
    obj_func = lambda trial: objective(trial, city)

    try:
      # Optimise over 100 trials
      study.optimize(obj_func, n_trials=ntry, n_jobs=njob)

      # Print results
      print("Study statistics for : ")
      print("  Number of finished trials: ", len(study.trials))
      
      print("Best trial of city: ",city)
      best_trial = study.best_trial
      print("  Value: ", best_trial.value)   

      # lưu best param vào trong biến toàn cục
      one_city_param = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'transformer',
                              'Best_value': best_trial.value,
                              'n_try_opt': ntry,
                              'n Feature': 3, # set cứng
                              'Batch Size': 16, # set cứng
                              'Lookback Window': 3, # set cứng
                              'Epochs': best_trial.params['Epochs'],
                              'Hidden Size': best_trial.params['Hidden size'],
                              'n Layers': best_trial.params['n layers'],
                              'Learning rate': best_trial.params['Learning rate'], 
                              'Num. filters': '', # Transformer không dùng
                              'Dropout rate': best_trial.params['Dropout rate']}, index=[0])
      one_city_param.to_excel(prj_path_opt+'/tf/diarrhoea_opt_hyperparam_transformer_'+city+'.xlsx')
      best_param = best_param.append(one_city_param)
    except:# có error thì lưu vào l_errCity để check lại sau 
      l_errCity.append(city)
    # Plot result
    l_studies[city] = study # thêm vào danh sách sài sau
    print('optimize result of city: '+ city)
    # optuna.visualization.plot_optimization_history(study)
    # optuna.visualization.plot_param_importances(study)
    # optuna.visualization.plot_slice(study)
  
  best_param.to_excel(prj_path_opt+'/tf/diarrhoea_opt_hyperparam_transformer.xlsx')

dt_ended = datetime.now()
print('kết thúc study trong:', round((dt_ended - dt_started).total_seconds()/60))
# print(l_errCity)

[I 2023-02-13 01:17:07,945] A new study created in memory with name: no-name-4bca0d53-3f7f-430c-a8e4-1824a369cd72


lookback 3
lookback 3
lookback 3
lookback 3
lookback 3
lookback 3
lookback 3
lookback 3
lookback 3
lookbacklookback 3
lookback 3
 3
lookback 3
lookback 3
lookback lookback 3
3
lookback 3
lookback 3
lookback 3
lookback 3
Epoch:  0/450 - train_loss: 1.2941 - test_loss: 0.925149
Epoch:  0/450 - train_loss: 1.4110 - test_loss: 1.154114
Epoch:  0/450 - train_loss: 1.5861 - test_loss: 0.794314
Epoch:  0/450 - train_loss: 1.1724 - test_loss: 0.766046
Epoch:  0/450 - train_loss: 0.9170 - test_loss: 0.659573
Epoch:  0/450 - train_loss: 1.4583 - test_loss: 1.026188
Epoch:  0/450 - train_loss: 1.2704 - test_loss: 0.794933
Epoch:  0/450 - train_loss: 0.9714 - test_loss: 0.980122Epoch:  0/450 - train_loss: 1.5842 - test_loss: 1.468478

Epoch:  0/450 - train_loss: 1.3174 - test_loss: 0.851732Epoch:  0/450 - train_loss: 1.8236 - test_loss: 2.877091

Epoch:  0/450 - train_loss: 1.7540 - test_loss: 1.794553
Epoch:  0/450 - train_loss: 1.0518 - test_loss: 0.818532
Epoch:  0/450 - train_loss: 1.3481 - te

[I 2023-02-13 01:39:29,565] Trial 7 finished with value: 26.219128771475308 and parameters: {'n layers': 6, 'Hidden size': 33, 'Learning rate': 0.0031659662891437884, 'Dropout rate': 0.718192633983721, 'Epochs': 300}. Best is trial 7 with value: 26.219128771475308.


lookback 3
Epoch: 448/450 - train_loss: 0.2847 - test_loss: 0.292776
Epoch: 449/450 - train_loss: 0.2937 - test_loss: 0.292923
Epoch: 336/450 - train_loss: 0.3061 - test_loss: 0.293112


[I 2023-02-13 01:39:40,951] Trial 16 finished with value: 31.14672702446497 and parameters: {'n layers': 4, 'Hidden size': 110, 'Learning rate': 0.0005385839819431862, 'Dropout rate': 0.15545238036392217, 'Epochs': 180}. Best is trial 7 with value: 26.219128771475308.


...

lookback 3
Epoch: 150/300 - train_loss: 0.3690 - test_loss: 0.170231
Epoch:  0/240 - train_loss: 1.1356 - test_loss: 0.727257
Epoch: 150/300 - train_loss: 0.3103 - test_loss: 0.187841
Epoch: 150/300 - train_loss: 0.3104 - test_loss: 0.179849
Epoch: 110/220 - train_loss: 0.3172 - test_loss: 0.182935
Epoch:  0/240 - train_loss: 0.8127 - test_loss: 0.662486
Epoch: 165/220 - train_loss: 0.3292 - test_loss: 0.213884
Epoch: 339/340 - train_loss: 0.3085 - test_loss: 0.181376


[I 2023-02-14 06:42:26,241] Trial 36 finished with value: 18.037895651820644 and parameters: {'n layers': 3, 'Hidden size': 112, 'Learning rate': 0.009859928958933851, 'Dropout rate': 0.4621532536146774, 'Epochs': 340}. Best is trial 30 with value: 17.505036682050086.


lookback 3
Epoch: 225/300 - train_loss: 0.3088 - test_loss: 0.164057
Epoch: 219/220 - train_loss: 0.3153 - test_loss: 0.183665
Epoch:  0/240 - train_loss: 1.1254 - test_loss: 0.713417
Epoch: 55/220 - train_loss: 0.3223 - test_loss: 0.191491


[I 2023-02-14 06:43:08,819] Trial 40 finished with value: 18.052833593439033 and parameters: {'n layers': 3, 'Hidden size': 108, 'Learning rate': 0.004166279246184093, 'Dropout rate': 0.4672757466771832, 'Epochs': 300}. Best is trial 30 with value: 17.505036682050086.


lookback 3
Epoch: 165/220 - train_loss: 0.3521 - test_loss: 0.178965
Epoch: 201/270 - train_loss: 0.3621 - test_loss: 0.179922
Epoch: 225/300 - train_loss: 0.3234 - test_loss: 0.173798
Epoch: 60/240 - train_loss: 0.3179 - test_loss: 0.182254
Epoch: 170/340 - train_loss: 0.3194 - test_loss: 0.200012
Epoch: 225/300 - train_loss: 0.3131 - test_loss: 0.171418


In [ ]:
import requests

def send_to_telegram(message):

    apiToken = '5908735099:AAGVSLrW62aXPBP-GrMvxoVgMsuJxXJpP1Q'
    chatID = '@ptn_announcement'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
        print(response.text)
    except Exception as e:
        print(e)

send_to_telegram("Server Chạy Xong TF" )